# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2bf4558760>
├── 'a' --> tensor([[-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832]])
└── 'x' --> <FastTreeValue 0x7f2bf4558c70>
    └── 'c' --> tensor([[ 0.5470, -0.2266, -0.3769,  1.9896],
                        [-1.6044,  0.4586,  0.3797, -0.7761],
                        [ 0.5929,  0.9527,  0.4309, -0.6402]])

In [4]:
t.a

tensor([[-0.2771,  0.5137, -0.0743],
        [-2.6599, -2.2271, -1.2832]])

In [5]:
%timeit t.a

73.6 ns ± 0.35 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2bf4558760>
├── 'a' --> tensor([[ 0.9485,  0.1010, -0.0490],
│                   [-1.2548,  0.2206, -1.1783]])
└── 'x' --> <FastTreeValue 0x7f2bf4558c70>
    └── 'c' --> tensor([[ 0.5470, -0.2266, -0.3769,  1.9896],
                        [-1.6044,  0.4586,  0.3797, -0.7761],
                        [ 0.5929,  0.9527,  0.4309, -0.6402]])

In [7]:
%timeit t.a = new_value

83.4 ns ± 0.313 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832]]),
    x: Batch(
           c: tensor([[ 0.5470, -0.2266, -0.3769,  1.9896],
                      [-1.6044,  0.4586,  0.3797, -0.7761],
                      [ 0.5929,  0.9527,  0.4309, -0.6402]]),
       ),
)

In [10]:
b.a

tensor([[-0.2771,  0.5137, -0.0743],
        [-2.6599, -2.2271, -1.2832]])

In [11]:
%timeit b.a

70.6 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7524,  0.4018, -0.5581],
               [-0.2780, -1.5766, -0.0258]]),
    x: Batch(
           c: tensor([[ 0.5470, -0.2266, -0.3769,  1.9896],
                      [-1.6044,  0.4586,  0.3797, -0.7761],
                      [ 0.5929,  0.9527,  0.4309, -0.6402]]),
       ),
)

In [13]:
%timeit b.a = new_value

571 ns ± 1.96 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

971 ns ± 4.95 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

163 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

162 µs ± 883 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2b50f04760>
├── 'a' --> tensor([[[-0.2771,  0.5137, -0.0743],
│                    [-2.6599, -2.2271, -1.2832]],
│           
│                   [[-0.2771,  0.5137, -0.0743],
│                    [-2.6599, -2.2271, -1.2832]],
│           
│                   [[-0.2771,  0.5137, -0.0743],
│                    [-2.6599, -2.2271, -1.2832]],
│           
│                   [[-0.2771,  0.5137, -0.0743],
│                    [-2.6599, -2.2271, -1.2832]],
│           
│                   [[-0.2771,  0.5137, -0.0743],
│                    [-2.6599, -2.2271, -1.2832]],
│           
│                   [[-0.2771,  0.5137, -0.0743],
│                    [-2.6599, -2.2271, -1.2832]],
│           
│                   [[-0.2771,  0.5137, -0.0743],
│                    [-2.6599, -2.2271, -1.2832]],
│           
│                   [[-0.2771,  0.5137, -0.0743],
│                    [-2.6599, -2.2271, -1.2832]]])
└── 'x' --> <FastTreeValue 0x7f2bcc19c2b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.9 µs ± 449 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2b4a22da00>
├── 'a' --> tensor([[-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832],
│                   [-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832],
│                   [-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832],
│                   [-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832],
│                   [-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832],
│                   [-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832],
│                   [-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832],
│                   [-0.2771,  0.5137, -0.0743],
│                   [-2.6599, -2.2271, -1.2832]])
└── 'x' --> <FastTreeValue 0x7f2b4a22d5e0>
    └── 'c' --> tensor([[ 0.5470, -0.2266, -0.3769,  1.9896],
                        [-1.6044,  0.4586,  0.3797, -0.7761],
                 

In [23]:
%timeit t_cat(trees)

35.1 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.7 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2771,  0.5137, -0.0743],
                [-2.6599, -2.2271, -1.2832]],
       
               [[-0.2771,  0.5137, -0.0743],
                [-2.6599, -2.2271, -1.2832]],
       
               [[-0.2771,  0.5137, -0.0743],
                [-2.6599, -2.2271, -1.2832]],
       
               [[-0.2771,  0.5137, -0.0743],
                [-2.6599, -2.2271, -1.2832]],
       
               [[-0.2771,  0.5137, -0.0743],
                [-2.6599, -2.2271, -1.2832]],
       
               [[-0.2771,  0.5137, -0.0743],
                [-2.6599, -2.2271, -1.2832]],
       
               [[-0.2771,  0.5137, -0.0743],
                [-2.6599, -2.2271, -1.2832]],
       
               [[-0.2771,  0.5137, -0.0743],
                [-2.6599, -2.2271, -1.2832]]]),
    x: Batch(
           c: tensor([[[ 0.5470, -0.2266, -0.3769,  1.9896],
                       [-1.6044,  0.4586,  0.3797, -0.7761],
                       [ 0.5929,  0.9527,  0.4309, -0.6402]],
         

In [26]:
%timeit Batch.stack(batches)

88.7 µs ± 668 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832],
               [-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832],
               [-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832],
               [-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832],
               [-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832],
               [-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832],
               [-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832],
               [-0.2771,  0.5137, -0.0743],
               [-2.6599, -2.2271, -1.2832]]),
    x: Batch(
           c: tensor([[ 0.5470, -0.2266, -0.3769,  1.9896],
                      [-1.6044,  0.4586,  0.3797, -0.7761],
                      [ 0.5929,  0.9527,  0.4309, -0.6402],
                      [ 0.5470, -0.2266, -0.3769,  1.9896],
                      [-1.6044,  

In [28]:
%timeit Batch.cat(batches)

164 µs ± 2.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

384 µs ± 9.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
